# Humpback Whale Identification - Keras Perceptron + B&W

In [1]:
import gc
import os

import numpy as np
import pandas as pd
import progressbar

import matplotlib.pyplot as plt
import matplotlib.image as mplimg
from matplotlib.pyplot import imshow

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout
from keras.models import Model

import keras.backend as K
from keras.models import Sequential

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

Using TensorFlow backend.


In [2]:
train_df = pd.read_csv("../input/train.csv")
train_df.head()

,Image,Id
0,0000e88ab.jpg,w_f48451c
1,0001f9222.jpg,w_c3d896a
2,00029d126.jpg,w_20df2c5
3,00050a15a.jpg,new_whale
4,0005c1ef8.jpg,new_whale


In [3]:
# yvan made this block, he's really proud of him
import cv2

def colors2BlackAndWhite(path):
    #-----Reading the image-----------------------------------------------------
    img = cv2.imread(path)
    #-----Black and White-------------------------------------------------------
    bw = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #-----Merge the CLAHE enhanced L-channel with the a and b channel-----------
    limg = cv2.merge((bw, bw, bw)) # (cl, a, b)
    #-----Converting image from LAB Color model to RGB model--------------------
    final = limg #final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
    #-----Resize----------------------------------------------------------------
    resized = cv2.resize(final, dsize=(128, 128), interpolation=cv2.INTER_CUBIC)
    return resized

In [4]:
def prepareImages(data, m, dataset):
    #print("Preparing images")
    #print(m)
    X_train = np.zeros((m, 128, 128, 3))
    count = 0
    
    for fig in progressbar.progressbar(data['Image']):
        # load images into images of size 128x128x3
        x = colors2BlackAndWhite("../input/"+dataset+"/"+fig) # added by Yvan
        x = preprocess_input(x)
        X_train[count] = x.flatten() # <------------------ I FIXED A BUG HERE
        count += 1
    
    return X_train

In [5]:
def prepare_labels(y):
    values = np.array(y)
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(values)
    #print(integer_encoded)

    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    #print(onehot_encoded)

    y = onehot_encoded
    #print(y.shape)
    return y, label_encoder

In [6]:
X = prepareImages(train_df, train_df.shape[0], "train")
X /= 255

MemoryError: 

In [ ]:
y, label_encoder = prepare_labels(train_df['Id'])

In [ ]:
y.shape

## Train

In [ ]:
model = Sequential()

model.add(Dense(64, activation='relu', input_shape = (128, 128, 3)))

model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(X, y, epochs=100, batch_size=100, verbose=1)
gc.collect()

In [ ]:
plt.plot(history.history['acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.show()

## Test

In [ ]:
test = os.listdir("../input/test/")
print(len(test))

In [ ]:
col = ['Image']
test_df = pd.DataFrame(test, columns=col)
test_df['Id'] = ''

In [ ]:
X = prepareImages(test_df, test_df.shape[0], "test")
X /= 255

In [ ]:
predictions = model.predict(np.array(X), verbose=1)

In [ ]:
for i, pred in enumerate(predictions):
    test_df.loc[i, 'Id'] = ' '.join(label_encoder.inverse_transform(pred.argsort()[-5:][::-1]))

In [ ]:
test_df.head(10)
test_df.to_csv('keras-perceptron-black-and-white.csv', index=False) #> Score = ?

In [ ]:
!kaggle competitions submit -c humpback-whale-identification -f "keras-perceptron-black-and-white.csv" -m "Keras Perceptron + B&W"